In [4]:
# ==============================================================================
# PROJECT: GUARDIAN - Priority-Override Perception Framework
# VERSION: 5.2 (Lane-Obstacle Override: Bottom-Center + Hazard Classes)
# DESCRIPTION:
# - Fixes "object is detected but mode stays NORMAL" by:
#   (1) Using Bottom-Center point (feet / contact with road) instead of bbox center
#   (2) Triggering CRITICAL for ANY hazard obstacle in lane ROI (person/vehicles/bikes)
#   (3) Lowering confidence thresholds for night videos
# ==============================================================================

import cv2
import time
import numpy as np
from ultralytics import YOLO

print("Initializing GUARDIAN v5.2 (Lane-Obstacle Override)...")

model_normal = YOLO('yolov8s.pt')   # balanced
model_safety = YOLO('yolov8n.pt')   # fastest

# -----------------------------
# CONFIG (tune if needed)
# -----------------------------
SAFETY_DEADLINE = 0.060

FAST_SCAN_IMGSZ = 256
FAST_SCAN_CONF  = 0.10   # lower for night

NORMAL_IMGSZ    = 320
NORMAL_CONF     = 0.10   # lower for night

OVERRIDE_CONF   = 0.10

# COCO hazard classes:
# 0 person, 1 bicycle, 2 car, 3 motorbike, 5 bus, 7 truck
HAZARD_CLASS_IDS = [0, 1, 2, 3, 5, 7]

def warm_up():
    print("Stabilizing system hardware...")
    dummy = np.zeros((640, 640, 3), dtype=np.uint8)
    for _ in range(3):
        _ = model_safety(dummy, imgsz=160, verbose=False)
    print("System Ready.")

# -----------------------------
# ROI + geometry helpers
# -----------------------------
def lane_roi_polygon(w, h):
    """
    Trapezoid ROI representing the road area ahead.
    IMPORTANT: Adjust these points if ROI is too low/high for your dashcam.
    """
    return np.array([
        (int(0.05*w), int(0.98*h)),
        (int(0.40*w), int(0.62*h)),
        (int(0.60*w), int(0.62*h)),
        (int(0.95*w), int(0.98*h))
    ], dtype=np.int32)

def point_in_polygon(pt, poly):
    return cv2.pointPolygonTest(poly, pt, False) >= 0

def bottom_center_of_box(x1, y1, x2, y2):
    """
    Bottom-center is more meaningful for lane occupancy:
    pedestrian feet / vehicle contact point on road.
    """
    return ((x1 + x2) / 2.0, y2)

def has_hazard_in_roi(results, roi_poly, conf_th=0.10, class_ids=None):
    """
    True if any hazard object exists inside ROI based on bottom-center point.
    """
    r = results[0]
    if r.boxes is None or len(r.boxes) == 0:
        return False, None

    boxes = r.boxes.xyxy.cpu().numpy()
    clss  = r.boxes.cls.cpu().numpy().astype(int)
    confs = r.boxes.conf.cpu().numpy()

    for (x1, y1, x2, y2), c, conf in zip(boxes, clss, confs):
        if conf < conf_th:
            continue
        if class_ids is not None and c not in class_ids:
            continue

        bc = bottom_center_of_box(x1, y1, x2, y2)
        if point_in_polygon(bc, roi_poly):
            return True, (int(c), float(conf), (x1, y1, x2, y2), bc)

    return False, None

def cls_name(c):
    return {
        0: "PERSON",
        1: "BICYCLE",
        2: "CAR",
        3: "MOTORBIKE",
        5: "BUS",
        7: "TRUCK",
    }.get(c, f"CLS{c}")

# -----------------------------
# Engine
# -----------------------------
def guardian_engine(frame, safety_deadline=SAFETY_DEADLINE):
    start_time = time.time()
    h, w = frame.shape[:2]
    roi_poly = lane_roi_polygon(w, h)

    # (A) FAST HAZARD SCAN in ROI (immediate)
    scan = model_safety(
        frame,
        imgsz=FAST_SCAN_IMGSZ,
        verbose=False,
        classes=HAZARD_CLASS_IDS,  # not only person: any obstacle in lane
        conf=FAST_SCAN_CONF
    )

    hazard_fast, info_fast = has_hazard_in_roi(scan, roi_poly, conf_th=FAST_SCAN_CONF, class_ids=HAZARD_CLASS_IDS)
    if hazard_fast:
        c, conf, box, bc = info_fast
        mode = f"CRITICAL: OBSTACLE IN PATH (FAST) [{cls_name(c)} {conf:.2f}]"
        return scan, time.time() - start_time, mode, roi_poly

    # (B) NORMAL inference
    results = model_normal(
        frame,
        imgsz=NORMAL_IMGSZ,
        verbose=False,
        conf=NORMAL_CONF
    )

    # (C) POST-OVERRIDE (most reliable): use SAME results that are plotted
    hazard_post, info_post = has_hazard_in_roi(results, roi_poly, conf_th=OVERRIDE_CONF, class_ids=HAZARD_CLASS_IDS)
    if hazard_post:
        c, conf, box, bc = info_post
        mode = f"CRITICAL: OBSTACLE IN PATH (OVERRIDE) [{cls_name(c)} {conf:.2f}]"
        return results, time.time() - start_time, mode, roi_poly

    # (D) Adaptive load (only if no hazard)
    latency_check = time.time() - start_time
    if latency_check > (safety_deadline * 0.75):
        mode = "ADAPTIVE: HIGH SYSTEM LOAD"
        results = model_safety(frame, imgsz=160, verbose=False, conf=0.15)
    else:
        mode = "NORMAL: CLEAR PATH"

    return results, time.time() - start_time, mode, roi_poly

# -----------------------------
# Video pipeline
# -----------------------------
def process_dashcam_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Path {input_path} is invalid.")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS) or 30.0

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    print(f"Analyzing: {input_path}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results, latency, mode, roi_poly = guardian_engine(frame)

        annotated_frame = results[0].plot()

        # Draw ROI
        cv2.polylines(annotated_frame, [roi_poly], True, (255, 255, 0), 2)

        # UI color
        is_normal = mode.startswith("NORMAL")
        status_color = (0, 255, 0) if is_normal else (0, 0, 255)

        cv2.rectangle(annotated_frame, (0, 0), (width, 90), (0, 0, 0), -1)
        cv2.putText(annotated_frame, f"STATUS: {mode}", (20, 35),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, status_color, 2)
        cv2.putText(annotated_frame, f"LATENCY: {latency:.4f}s", (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        out.write(annotated_frame)

    cap.release()
    out.release()
    print(f"Analysis Finished. File: {output_path}")

# --- EXECUTION ---
warm_up()
INPUT_FILE = "T2.MP4"
OUTPUT_FILE = "GUARDIAN_v5_2_LaneObstacleOverride.mp4"
process_dashcam_video(INPUT_FILE, OUTPUT_FILE)

Initializing GUARDIAN v5.2 (Lane-Obstacle Override)...
Stabilizing system hardware...
System Ready.
Analyzing: T2.MP4
Analysis Finished. File: GUARDIAN_v5_2_LaneObstacleOverride.mp4
